In [1]:
import os
path = "C:/Users/mghn_/Desktop/Imarticus_Project"
os.chdir(path)
print(os.getcwd())

C:\Users\mghn_\Desktop\Imarticus_Project


In [2]:
print(os.getcwd())

C:\Users\mghn_\Desktop\Imarticus_Project


In [3]:
import pandas as pd
import numpy as np

data=pd.read_csv("TrainData_CSM.csv")

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [5]:
data.head(3)

,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,...,total_rec_late_fee,recoveries,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,default_ind
0,5000.0,0,10.65,1,21,150119,10.0,2,24000.0,2,...,0.0,0.0,171.62,0.0,1.0,0,0.0,0.0,81008.5,0
1,2500.0,1,15.27,2,33,186161,0.5,2,30000.0,1,...,0.0,0.0,119.66,0.0,1.0,0,0.0,0.0,81008.5,1
2,2400.0,0,15.96,2,34,150119,10.0,2,12252.0,0,...,0.0,0.0,649.91,0.0,1.0,0,0.0,0.0,81008.5,0


In [6]:
data.drop("issue_d",axis=1,inplace=True)

In [7]:
y=data['default_ind']


In [8]:
X=data.drop('default_ind',axis=1)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
#First split criterion used in Gini Index

In [11]:
rf=RandomForestClassifier(random_state=1)

In [12]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [13]:
y_pred=rf.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

print("----------Using Gini index (Default)-----------")
print("Classification Report:\n",classification_report(y_test,y_pred))
print("Confusion Matrix \n",confusion_matrix(y_test,y_pred))
acc_score=accuracy_score(y_test,y_pred)
print("The test accuracy is {:.3f}",round(acc_score,5))



----------Using Gini index (Default)-----------
Classification Report:
              precision    recall  f1-score   support

          0       0.99      1.00      0.99     88258
          1       0.99      0.89      0.94      8974

avg / total       0.99      0.99      0.99     97232

Confusion Matrix 
 [[88179    79]
 [  961  8013]]
The test accuracy is {:.3f} 0.9893


In [15]:
testData=pd.read_csv("TestData_CSM.csv")

In [16]:
testData.drop('issue_d',axis=1,inplace=True)

In [17]:
y=testData['default_ind']
X=testData.drop('default_ind',axis=1)

In [18]:
y_testData=rf.predict(X)

In [19]:
print("----------Using Gini on Test Data-----------")
print("Classification Report:\n",classification_report(y,y_testData))
print("Confusion Matrix \n",confusion_matrix(y, y_testData))
acc_score=accuracy_score(y,y_testData)
acc_score_test=accuracy_score(y,y_testData)
print("The test accuracy is {:.3f}",round(acc_score_test,5))

----------Using Gini on Test Data-----------
Classification Report:
              precision    recall  f1-score   support

          0       1.00      0.82      0.90    367646
          1       0.03      0.91      0.06      2164

avg / total       0.99      0.82      0.90    369810

Confusion Matrix 
 [[301105  66541]
 [   196   1968]]
The test accuracy is {:.3f} 0.81954


In [37]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
cols = ['Model', 'ROC Score', 'Precision Score', 'Recall Score','Accuracy Score']
models_report = pd.DataFrame(columns = cols)
tmp1 = pd.Series({'Model': " Random Forest Report",
                 'ROC Score' : roc_auc_score(y,y_testData),
                 'Precision Score': precision_score(y,y_testData),
                 'Recall Score': recall_score(y,y_testData),
                 'Accuracy Score': accuracy_score(y,y_testData)})
model1_report = models_report.append(tmp1, ignore_index = True)
model1_report

,Model,ROC Score,Precision Score,Recall Score,Accuracy Score
0,Random Forest Report,0.864217,0.028726,0.909427,0.819537


In [20]:
#Model tuning Begins

In [21]:
print(rf.get_params)

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)>


In [22]:
#Model tuning on Gini index of TrainData.CSV
from sklearn.model_selection import GridSearchCV

In [26]:
params_rf={'max_depth':[3,4,5,6], 'min_samples_leaf':[0.03,0.04,0.05,0.06],'max_features':[0.2,0.4,0.6]}

In [28]:
grid_rf=GridSearchCV(estimator=rf,param_grid=params_rf,scoring='accuracy',cv=10,n_jobs=-1)

In [29]:
rf_cv=grid_rf.fit(X_train,y_train)

In [30]:
best_hyperparam=rf_cv.best_params_
print("Best parameters :",best_hyperparam)

Best parameters : {'max_depth': 4, 'max_features': 0.6, 'min_samples_leaf': 0.03}


In [31]:
best_cv_score=rf_cv.best_score_
print("Best CV score :",best_cv_score)

Best CV score : 0.9433184119384872


In [32]:
best_model=rf_cv.best_estimator_
print("The best estimators ",rf_cv.best_estimator_)

The best estimators  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features=0.6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=0.03, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)


In [33]:
#Trying to compute accuracy on validation set
test_accuracy=best_model.score(X_test,y_test)

In [34]:
print("Best accuracy",test_accuracy)

Best accuracy 0.9399168997860786


In [35]:
#Testing on training data
testData_accuracy=best_model.score(X,y)
print("Performance on unseen data",testData_accuracy)

Performance on unseen data 0.9961385576377059
